In [1]:
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('player-clustering').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

In [2]:
batsman_cluster=sqlContext.read.format('parquet').load('batsman_cluster')
bowler_cluster=sqlContext.read.format('parquet').load('bowler_cluster')

In [3]:
import pandas as pd
batsman_cluster = batsman_cluster.toPandas()
bowler_cluster = bowler_cluster.toPandas()
name_map = dict()

In [4]:
batsman_cluster.to_csv('batsman_cluster.csv',index=False)
bowler_cluster.to_csv('bowler_cluster.csv',index=False)
new_batsman_cluster = batsman_cluster.set_index('player_name')
new_bowler_cluster = bowler_cluster.set_index('player_name')

In [5]:
ipl = pd.read_csv('new.csv',index_col='key')
# print(ipl.head())
# k=ipl.replace({'key':r'P* Ojha, *'},{'key':'1, '},regex=True)
# k.head()
print(ipl.filter(regex='^P* Ojha, R* Bhatia',axis=0))



                    0s   1s   2s   3s   4s   6s  outs  balls
key                                                         
PP Ojha, R Bhatia  1.0  1.0  1.0  1.0  1.0  1.0   0.5      2


In [6]:
#gets runs batsman a and bowler b
def pred(a,b,random):
    k='^'+a+', '+b
    print(k)
    cum_prob = ipl.filter(regex=k,axis=0)
    print(cum_prob)
    try:
        for i in range(0,6):
            if((random<=cum_prob.iloc[:,i]).bool()):
                if(i==5):
                    return 6
                else:
                    return i
    except:
        pass

In [7]:
#makes a regular expression in the form required using passed arg
def make_reg(a):
    a=a.split()
    reg=a[0][0]+'.*'+a[-1]
    return reg

In [8]:
#showing use case
t = pred(make_reg('A Ashish Reddy'), make_reg('DJ Bravo'),0.1)

^A.*Reddy, D.*Bravo
                                0s        1s        2s        3s        4s  \
key                                                                          
A Ashish Reddy, DJ Bravo  0.307692  0.615385  0.692308  0.692308  0.923077   

                           6s      outs  balls  
key                                             
A Ashish Reddy, DJ Bravo  1.0  0.076923     13  


In [9]:
#working of make_reg
make_reg('A Ashish Reddy')

'A.*Reddy'

In [10]:
temp = pd.read_csv('cluster.csv')

In [11]:
#replacing batsman names by cluster number
print(len(batsman_cluster))
count = 0
inter = temp
for i in range(len(batsman_cluster)):
    clust = batsman_cluster.iloc[i,11]
    name = batsman_cluster.iloc[i,0]
    k = '^'+make_reg(name)+','
    k2 = name+','
    found = new_batsman_cluster.filter(regex = make_reg(name), axis = 0)
    if len(found)!=1:
        found1 = new_batsman_cluster.filter(regex = name, axis = 0)
        new=inter.replace({'key':k2},{'key':str(clust)+','},regex=True)
        if len(found1)!=1:
            print(name,len(found))
    else:
        new=inter.replace({'key':k},{'key':str(clust)+','},regex=True)
    if(len(new[(new==inter)['key']==False])==0):
        k3 = '.*'+make_reg(name)+','
        new=new.replace({'key':k3},{'key':str(clust)+','},regex=True)
        if(len(new[(new==inter)['key']==False])==0):
#             print(name,k,clust)
            count+=1
    inter=new
print(count)

573
Amit Mishra 2
Amit Mishra 2
Jaskaran Singh 3
Aniket Choudhary 2
Jaskaran Singh 3
Aniket Choudhary 2
138


In [12]:
inter

,key,0s,1s,2s,3s,4s,6s,outs,balls
0,"3, R Bhatia",2,0,0,0,0,0,1,2
1,"0, R McLaren",1,1,1,0,0,0,0,3
2,"0, Iqbal Abdulla",2,5,0,0,0,0,0,7
3,"0, CH Morris",4,4,1,0,3,0,0,12
4,"3, A Nehra",3,2,1,0,2,0,0,8
5,"2, A Symonds",6,3,2,0,2,1,0,14
6,"0, AB Agarkar",1,2,0,0,2,1,0,6
7,"2, YS Chahal",2,2,0,0,0,1,0,5
8,"1, S Nadeem",3,3,0,0,1,0,0,7
9,"2, IK Pathan",0,4,1,0,2,0,0,7


In [13]:
#replacing bowler names by cluster number
print(len(bowler_cluster))
count = 0
inter = new
for i in range(len(bowler_cluster)):
    clust = bowler_cluster.iloc[i,7]
    name = bowler_cluster.iloc[i,0]
    k = ', '+make_reg(name)
    k2 = ', '+name
    found = new_bowler_cluster.filter(regex = make_reg(name), axis = 0)
    if len(found)!=1:
        found1 = new_bowler_cluster.filter(regex = name, axis = 0)
        new=inter.replace({'key':k2},{'key':', '+str(clust)},regex=True)
        if len(found1)!=1:
            print(name,len(found))
    else:
        new=inter.replace({'key':k},{'key':', '+str(clust)},regex=True)
    if(len(new[(new==inter)['key']==False])==0):
        k3 = ', .*'+make_reg(name)
        new=new.replace({'key':k3},{'key':', '+str(clust)},regex=True)
        if(len(new[(new==inter)['key']==False])==0):
        #         print(name,k,clust)
            count+=1
    inter=new
print(count)

606
Amit Mishra 2
Amit Mishra 2
Jaskaran Singh 3
Aniket Choudhary 2
Harmeet Singh 4
Harmeet Singh 4
Jaskaran Singh 3
Aniket Choudhary 2
279


In [14]:
inter

,key,0s,1s,2s,3s,4s,6s,outs,balls
0,"3, 4",2,0,0,0,0,0,1,2
1,"0, 2",1,1,1,0,0,0,0,3
2,"0, 4",2,5,0,0,0,0,0,7
3,"0, 2",4,4,1,0,3,0,0,12
4,"3, 4",3,2,1,0,2,0,0,8
5,"2, 3",6,3,2,0,2,1,0,14
6,"0, 3",1,2,0,0,2,1,0,6
7,"2, 4",2,2,0,0,0,1,0,5
8,"1, 4",3,3,0,0,1,0,0,7
9,"2, 2",0,4,1,0,2,0,0,7


In [15]:
inter.to_csv('temp.csv',index=False)

In [16]:
new=pd.read_csv('temp.csv')

In [17]:
new=new[new.key.str.match('[0-5], [0-5]')]

In [18]:
new

,key,0s,1s,2s,3s,4s,6s,outs,balls
0,"3, 4",2,0,0,0,0,0,1,2
1,"0, 2",1,1,1,0,0,0,0,3
2,"0, 4",2,5,0,0,0,0,0,7
3,"0, 2",4,4,1,0,3,0,0,12
4,"3, 4",3,2,1,0,2,0,0,8
5,"2, 3",6,3,2,0,2,1,0,14
6,"0, 3",1,2,0,0,2,1,0,6
7,"2, 4",2,2,0,0,0,1,0,5
8,"1, 4",3,3,0,0,1,0,0,7
9,"2, 2",0,4,1,0,2,0,0,7


In [19]:
#grouping ball by ball cluster info and summing
temp=new.groupby('key')

In [20]:
temp=temp.sum()

In [21]:
temp=temp.drop(('1, 3 (2)'))
temp=temp.drop(('2, 3 (2)'))


In [22]:
#finding mean
for i in range(7):
    temp.iloc[:,i]=temp.iloc[:,i]/temp.iloc[:,7]

In [23]:
temp.to_csv('final_cluster.csv')

In [24]:
temp

,0s,1s,2s,3s,4s,6s,outs,balls
key,,,,,,,,
"0, 0",0.282051,0.384615,0.102564,0.000000,0.128205,0.102564,0.051282,39
"0, 1",0.307692,0.461538,0.046154,0.000000,0.107692,0.076923,0.046154,65
"0, 2",0.430891,0.365641,0.069360,0.003007,0.091711,0.039290,0.063747,9977
"0, 3",0.415987,0.370106,0.068719,0.003263,0.096860,0.044657,0.059951,4904
"0, 4",0.412813,0.374867,0.070472,0.003450,0.096345,0.041725,0.057002,12175
"1, 0",0.346154,0.423077,0.038462,0.000000,0.115385,0.076923,0.038462,26
"1, 1",0.454545,0.272727,0.090909,0.000000,0.090909,0.090909,0.090909,11
"1, 2",0.440955,0.321196,0.054785,0.003174,0.128278,0.051445,0.039753,5987
"1, 3",0.406209,0.321664,0.058124,0.002312,0.136063,0.075297,0.038639,3028
